# G-квадруплексы

In [2]:
import re
from Bio import SeqIO
import numpy as np
import pandas as pd

In [3]:
input_file = "genomic.fna"
fasta_sequence = SeqIO.parse(input_file,'fasta')

pattern_plus = "((?:G{3,}[ATGC]{1,7}){3,}G{3,})"
pattern_minus = "((?:C{3,}[ATGC]{1,7}){3,}C{3,})"

In [4]:
results = pd.DataFrame(columns = ['Scaffold', 'Start', 'End', 'Name', 'Score', 'Strand'])

for record in SeqIO.parse(input_file, "fasta"):
    name, sequence = record.id, str(record.seq)
    PQS_plus=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern_plus, sequence, re.IGNORECASE)]
    PQS_plus_numpy = np.array(PQS_plus)
    if len(PQS_plus_numpy) == 0:
        continue
    new = pd.DataFrame(data={'Scaffold': [name]*len(PQS_plus_numpy),
                             'Start': PQS_plus_numpy[:,0], 'End': PQS_plus_numpy[:,1],
                             'Name': ['']*len(PQS_plus_numpy), 'Score': ['']*len(PQS_plus_numpy),
                             'Strand': ['+']*len(PQS_plus_numpy)})
    results = pd.concat([results, new], ignore_index=True)

    PQS_minus=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern_minus, sequence, re.IGNORECASE)]
    PQS_minus_numpy = np.array(PQS_minus)
    if len(PQS_minus_numpy) == 0:
        continue
    new = pd.DataFrame(data={'Scaffold': [name]*len(PQS_minus_numpy),
                             'Start': PQS_minus_numpy[:,0] , 'End': PQS_minus_numpy[:,1],
                             'Name': ['']*len(PQS_minus_numpy), 'Score': ['']*len(PQS_minus_numpy),
                             'Strand': ['-']*len(PQS_minus_numpy)})
    results = pd.concat([results, new], ignore_index=True)
results

,Scaffold,Start,End,Name,Score,Strand
0,NC_061452.1,11992,12012,,,+
1,NC_061452.1,31414,31439,,,+
2,NC_061452.1,80239,80256,,,+
3,NC_061452.1,118441,118468,,,+
4,NC_061452.1,131833,131854,,,+
...,...,...,...,...,...,...
16125,NW_025899719.1,13954,13988,,,-
16126,NW_025899719.1,14970,15002,,,-
16127,NW_025899721.1,16075,16091,,,+
16128,NW_025899721.1,4355,4385,,,-


In [5]:
results.to_csv('G-quadruplex.bed', index=False, header=False, sep='\t')